### My first opportunity using the Plotly visualization library, I use it here to create a Choropleth map showing the sources of visitors to Scottsdale that get arrested while visiting.

In [48]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import plotly 
plotly.tools.set_credentials_file(username='mrk.jhsn', api_key='S6wMeE9qKcvjDvncAyT2')

import numpy as np
import pyodbc
import pandas as pd

# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

#SQL Query pulls into Scottsdale arrestees with AZ arrestees already excluded 
ArresteesbyState = '''
 select  
	SUBSTRING(
		[City of Arrestee]
		,charindex(',',[City of Arrestee]) +2
		,2
	) as State_of_Arrestee
	,count(*) as _Count_
from [dbo].[spd_PDArrests$]
where [City of Arrestee] not like '%, AZ %' and --AZ arrestees excluded
	[City of Arrestee] like '%, __ _____' --messy, non-standard records excluded
group by SUBSTRING(
		[City of Arrestee]
		,charindex(',',[City of Arrestee]) +2
		,2
	)
    '''
# save the sql query to a pandas dataframe
ArresteesbyState = pd.read_sql(ArresteesbyState, connection)
connection.close()

#Create column 'Percent'
ArresteesbyState['Percent'] = ArresteesbyState._Count_.apply(lambda x: x/ArresteesbyState._Count_.sum()*100)

#format 'Percent_Arrested" with more decimal places
pd.options.display.float_format = '{:,.4f}'.format

ArresteesbyState['Percent'] = ArresteesbyState.Percent.astype(float)

#sort by states with the most arrestees
ArresteesbyState.sort_values(['_Count_'],ascending=False, inplace=True)

# rename columns for clarity
ArresteesbyState = ArresteesbyState.rename({'State_of_Arrestee': 'State', 'Percent': 'Percent_Arrested'}, axis= 'columns')

In [50]:
ArresteesbyState.head()

,State,_Count_,Percent_Arrested
5,CA,88,36.5145
11,IL,17,7.0539
33,TX,14,5.8091
36,WA,14,5.8091
24,NV,9,3.7344


In [61]:
data = [go.Choropleth(   
    locations = ArresteesbyState.iloc[:,0],
    z = ArresteesbyState.iloc[:,1],
    locationmode = 'USA-states',
    text = ArresteesbyState.iloc[:,0])]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Source of Visitor Arrestees to Scottsdale AZ'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa')))

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'd3-cloropleth-map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~mrk.jhsn/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'
